<a href="https://colab.research.google.com/github/VigneshMohan-Git/models/blob/master/Copy_of_upgrade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

### Setup

Before getting started ensure that TensorlFlow 2.0 is installed.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
try:
  import tensorflow.compat.v2 as tf
except Exception:
  pass

tf.enable_v2_behavior()

print(tf.__version__)

Clone the [tensorflow/models](https://github.com/tensorflow/models) git repository so you have some code to test on:

In [0]:
!git clone --branch r1.13.0 --depth 1 https://github.com/tensorflow/models


### Read the help

The script should be installed with TensorFlow. Here is the builtin help:

In [0]:
!tf_upgrade_v2 -h

### Example TF1 code

Here is a simple TensorFlow 1.0 script:

In [0]:
!head -n 65 models/samples/cookbook/regression/custom_regression.py | tail -n 10

With TensorFlow 2.0 installed it does not run:

In [0]:
!(cd models/samples/cookbook/regression && python custom_regression.py)

### Single file

The upgrade script can be run on a single Python file:

In [0]:
!tf_upgrade_v2 \
  --infile models/samples/cookbook/regression/custom_regression.py \
  --outfile /tmp/custom_regression_v2.py

The script will print errors if it can not find a fix for the code. 

### Directory tree

Typical projects, including this simple example, will use much more than one file. Typically want to upgrade an entire package, so the script can also be run on a directory tree:

In [0]:
# upgrade the .py files and copy all the other files to the outtree
!tf_upgrade_v2 \
    --intree models/samples/cookbook/regression/ \
    --outtree regression_v2/ \
    --reportfile tree_report.txt

Note the one warning about the `dataset.make_one_shot_iterator` function.

Now the script works in with TensorFlow 2.0:

Note that because the `tf.compat.v1` module, the converted script will also run in TensorFlow 1.14. 

In [0]:
!(cd regression_v2 && python custom_regression.py 2>&1) | tail

## Detailed report

The script also reports a list of detailed changes. In this example it found one possibly unsafe transformation and included a warning at the top of the file: 

In [0]:
!head -n 20 tree_report.txt

Note again the one warning about the `Dataset.make_one_shot_iterator function`.

In other cases the output will explain the reasoning for non-trivial changes:

In [0]:
%%writefile dropout.py
import tensorflow as tf

d = tf.nn.dropout(tf.range(10), 0.2)
z = tf.zeros_like(d, optimize=False)

In [0]:
!tf_upgrade_v2 \
  --infile dropout.py \
  --outfile dropout_v2.py \
  --reportfile dropout_report.txt > /dev/null

In [0]:
!cat dropout_report.txt

Here is the modified file contents, note how the script adds argument names to deal with moved and renamed arguments:

In [0]:
!cat dropout_v2.py

A larger project might contain a few errors. For example convert the deeplab model:

In [0]:
!tf_upgrade_v2 \
    --intree models/research/deeplab \
    --outtree deeplab_v2 \
    --reportfile deeplab_report.txt > /dev/null

It produced the output files:

In [0]:
!ls deeplab_v2

But there were errors. The report will help you pin-point what you need to fix before this will run. Here are the first three errors:

In [0]:
!cat deeplab_report.txt | grep -i models/research/deeplab | grep -i error | head -n 3

## "Safety" mode

The conversion script also has a less invasive `SAFETY` mode that simply changes the imports to use the `tensorflow.compat.v1` module:

In [0]:
!cat dropout.py

In [0]:
!tf_upgrade_v2 --mode SAFETY --infile dropout.py --outfile dropout_v2_safe.py > /dev/null

In [0]:
!cat dropout_v2_safe.py